In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse.linalg import expm
from networkx.algorithms.traversal.breadth_first_search import bfs_edges
import json

from matplotlib import rcParams
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
rcParams['font.family'] = 'serif'
rcParams['mathtext.fontset'] = 'cm'
rcParams['figure.dpi'] = 100
rcParams['legend.fontsize'] = 10
rcParams['axes.labelsize'] = 'large'

In [3]:
with open("../proc_jsons.json", "r") as f:
    deps = json.load(f)

In [4]:
G = nx.DiGraph()
nodes = [d["Name"] for d in deps]
edges = []
for d in deps:
    edges += [(p, d["Name"]) for p in d["Dependencies"]]

G.add_nodes_from(nodes)
G.add_edges_from(edges)

del(deps)
del(nodes)
del(edges)

In [5]:
try:
    print("Found cycle:", nx.find_cycle(G, source="seaborn"))
except nx.NetworkXNoCycle:
    print("No cycle found")

Found cycle: [('Bitmasher', 'Bitmasher')]


In [ ]:
# Nodos con grado in 0 = Nodos que no dependen de nada.
no_dep_nodes = [x[0] for x in list(filter(lambda x: x[1] == 0, tuple(G.in_degree())))]

class SearchScope:

    def __init__(self, G) -> None:
        self.G = G
        self.scoped_nodes = {}

    def __call__(self, node: str, path: list = None) -> None:
        """
        Search scope of a node.

        Args:
            node (str): Node to search.
            path (list, optional): List of nodes reached before. Defaults to None.
        """
        
        neighs = list(nx.neighbors(self.G, node))
        scope = set()
        if path is None:
            path = []

        for n in neighs:
            if n not in path:
                try:
                    partial_scope = self.scoped_nodes[n]
                except KeyError:
                    if node not in path:
                        path.append(node)
                    self(n, path = path)
                    partial_scope = self.scoped_nodes[n]
                finally:
                    scope = scope | partial_scope | set([n])

        self.scoped_nodes[node] = scope
        print(len(self.scoped_nodes), end="\r")
    
        return None

search = SearchScope(G)

In [7]:
for n in no_dep_nodes:
    search(n)

: 

: 